# Projet Python/Algorithmique
Ce programme traite des fichiers au format CONLLU pour générer des statistiques détaillées. Il inclut des fonctionnalités pour la déduplication des fragments de N-Grammes, l'extraction de motifs de fréquence, et la génération de Skip-Grammes, en répondant aux exigences spécifiques de la partie individuelle du projet.

Importation des bibliothèques nécessaires.

In [37]:
import json
from collections import Counter, defaultdict

Fonction de lecture du fichier CONLLU.

In [38]:
def read_conllu(file_name):
    # On initialise une liste vide qui contiendra toutes les phrases
    # et une liste temporaire pour stocker les lignes de la phrase courante
    sentences = []
    current_sentence = []

    # Ouverture du fichier en lecture avec encodage UTF-8
    with open(file_name, 'r', encoding='utf-8') as file:
        for line in file:
            # On enlève les espaces et retours à la ligne superflus
            line = line.strip()

            # On ignore les lignes de commentaires
            if line.startswith('# text'):
                continue

            if not line:
                # Si on a des lignes dans la phrase courante,
                # on ajoute la phrase à notre liste de phrases
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []

            # Analyse la ligne et ajoute les infos pertinentes à la phrase courante si valide.
            else:
                parts = line.split('\t')
                if len(parts) >= 4:
                    current_sentence.append({
                        "id": parts[0],
                        "form": parts[1],
                        "lemma": parts[2],
                        "upos": parts[3],
                        "deprel": parts[7] if len(parts) > 7 else None
                    })
    # Pour ne pas oublier la dernière phrase si le fichier ne se termine pas par une ligne vide
    if current_sentence:
        sentences.append(current_sentence)
    return sentences

##### La fonction de validation.

In [39]:
def validate_conllu(sentences):
    # On s'assure d'avoir un fichier CONLLU fiable
    for sentence in sentences:
        for token in sentence:
            if not all(key in token for key in ["id", "form", "lemma", "upos"]):
                raise ValueError("CONLLU n'est pas trouvé ou est endommagé")

##### Traitement des Phrases
On analyse les statistiques d'un ensemble de phrases au format CONLLU. La fonction prend en paramètre une liste de phrases (output de read_conllu) et retourne un hachage contenant les statistiques demandées.

In [40]:
def process_sentences(sentences):
    """
    Analyse les statistiques d'un ensemble de phrases au format CONLLU.
    Prend en paramètre une liste de phrases (output de read_conllu).
    Retourne un hachage avec les statistiques demandées.
    """
    # Initialisation des compteurs
    nb_toks = 0
    nb_sents = len(sentences)
    forms = []
    lemmas = []
    nb_puncts = 0
    pos_counts = defaultdict(Counter)

    for sentence in sentences:
        # On compte le nombre de tokens
        nb_toks += len(sentence)
        for token in sentence:
            # On compte le nombre de ponctuations
            if token["upos"] == "PUNCT":
                nb_puncts += 1
                continue
            # On récupère les formes, les lemmes et les POS
            forms.append(token["form"])
            lemmas.append(token["lemma"])
            pos_counts[token["upos"]][token["lemma"]] += 1

    # On compte le nombre de formes et de types
    nb_forms = len(forms)
    nb_types = len(set(forms))
    # Calcul des moyennes
    average_sent_length = nb_toks / nb_sents
    average_form_length = sum(len(form) for form in forms) / nb_forms

    # Création du hachage de retour
    return {
        "nbToks": nb_toks,
        "nbSents": nb_sents,
        "nbForms": nb_forms,
        "nbPuncts": nb_puncts,
        "nbTypes": nb_types,
        "averageSentLength": average_sent_length,
        "averageFormLength": average_form_length,
        # Conversion des hachages en tableaux triés par fréquence
        "noun2freq": sorted(pos_counts["NOUN"].items(), key=lambda x: x[1], reverse=True),
        "verb2freq": sorted(pos_counts["VERB"].items(), key=lambda x: x[1], reverse=True),
        "adj2freq": sorted(pos_counts["ADJ"].items(), key=lambda x: x[1], reverse=True),
        "adv2freq": sorted(pos_counts["ADV"].items(), key=lambda x: x[1], reverse=True),
        "lem2freq": sorted(Counter(lemmas).items(), key=lambda x: x[1], reverse=True)
    }

##### Génération des N-Grammes
- Filtrage des Lemmes
- Calcul des N-Grammes
- Tri par fréquence

In [41]:
def generate_ngrams(lemmas, min_len=2, max_len=6):
    # Calcul des n-grammes
    lemmas = [lemma for lemma in lemmas if lemma not in [",", ".", "!", "?", ":", ";", "-", "_", "(", ")"]]

    ngrams = defaultdict(list)
    for n in range(min_len, max_len + 1):
        for i in range(len(lemmas) - n + 1):
            ngram = tuple(lemmas[i:i + n])
            ngrams[n].append(ngram)
    # Tri par fréquence
    return {key: sorted(Counter(value).items(), key=lambda x: x[1], reverse=True) for key, value in ngrams.items()}


##### Dédoublenage des N-Grammes

In [42]:
# Supprime les n-grammes plus courts redondants si leur fréquence est inférieure à un seuil.
def deduplicate_ngrams(ngrams, seuil_dedoublonnage):
    for n in sorted(ngrams.keys(), reverse=True):
        if n - 1 not in ngrams:
            continue
        for longer_ngram, freq_longer in ngrams[n]:
            shorter_candidates = [longer_ngram[:i] + longer_ngram[i + 1:] for i in range(len(longer_ngram))]
            for shorter_ngram in shorter_candidates:
                for idx, (short_ngram, freq_short) in enumerate(ngrams[n - 1]):
                    if short_ngram == shorter_ngram and freq_short <= seuil_dedoublonnage * freq_longer:
                        ngrams[n - 1].pop(idx)
                        break
    return ngrams


##### Extraction des Motifs

In [43]:
# Extrait et compte les fréquences des séquences de lemmes correspondant à des motifs POS donnés.
def extract_patterns(sentences, patterns):
    pattern_freqs = Counter()
    for sentence in sentences:
        lemmas = [token["lemma"] for token in sentence]
        pos_tags = [token["upos"] for token in sentence]
        for pattern in patterns:
            pattern_len = len(pattern)
            for i in range(len(lemmas) - pattern_len + 1):
                if pos_tags[i:i + pattern_len] == pattern:
                    pattern_freqs[" ".join(lemmas[i:i + pattern_len])] += 1
    return sorted(pattern_freqs.items(), key=lambda x: x[1], reverse=True)


##### Génération des Skip-Grammes à partir d'une liste des lemmes.
Elle prend en compte un gap défini entre les mots et génère des séquences de longueur comprise entre min_len et max_len. Les skipgrammes sont ensuite triés par fréquence décroissante.

In [44]:
def generate_skipgrams(lemmas, gap=1, min_len=2, max_len=3):
    skipgrams = defaultdict(int)
    n = len(lemmas)

    for length in range(min_len, max_len + 1):
        for i in range(n - length + 1):
            skipgram = tuple(lemmas[i:i + length:gap])
            skipgrams[skipgram] += 1

    return sorted(skipgrams.items(), key=lambda x: x[1], reverse=True)

 # SKIPGRAMS SANS PUNCT

 # def generate_skipgrams(lemmas, gap=1, min_len=2, max_len=3):
 #    lemmas = [lemma for lemma in lemmas if lemma not in [",", ".", "!", "?", ":", ";", "-", "_", "(", ")"]]
 #
 #    skipgrams = defaultdict(int)
 #    n = len(lemmas)
 #
 #    for length in range(min_len, max_len + 1):
 #        for i in range(n - length + 1):
 #            skipgram = tuple(lemmas[i:i + length:gap])
 #            skipgrams[skipgram] += 1
 #
 #    return sorted(skipgrams.items(), key=lambda x: x[1], reverse=True)



##### Fonction Principale

In [45]:
def main(conllu_file, json_file, pattern_file=None, seuil_dedoublonnage=1.3, ngram_min=2, ngram_max=6, skipgram_gap=1):
    # Lecture et validation du fichier CoNLL-U
    sentences = read_conllu(conllu_file)
    validate_conllu(sentences)
    stats = process_sentences(sentences)

    # Génération des n-grammes
    lemmas = [token["lemma"] for sentence in sentences for token in sentence]
    ngrams = generate_ngrams(lemmas, ngram_min, ngram_max)
    stats["ngrams"] = deduplicate_ngrams(ngrams, seuil_dedoublonnage)

    # Extraction des motifs si un fichier de motifs est fourni
    if pattern_file:
        try:
            with open(pattern_file, 'r', encoding='utf-8') as pf:
                patterns = json.load(pf)
            stats["patterns"] = extract_patterns(sentences, patterns)
        except FileNotFoundError:
            print(f"Fichier de patterns '{pattern_file}' introuvable.")

    # Génération des skip-grammes
    stats["skipgrams"] = generate_skipgrams(lemmas, gap=skipgram_gap, min_len=ngram_min, max_len=ngram_max)

    # Écriture des résultats dans un fichier JSON
    with open(json_file, 'w', encoding='utf-8') as json_out:
        json.dump(stats, json_out, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    conllu_file_path = "./fr_rhapsodie-ud-test.conllu"
    json_file_path = "results.json"
    pattern_file_path = "patterns.json"

    main(conllu_file_path, json_file_path, pattern_file=pattern_file_path)

### PSEUDO-CODE 
pour l'algorithme de calcul des index hiérarchiques des N-Grammes.

L'algorithme a une complexité QUADRATIQUE. Elle est déterminée par les fonctions générerNgrammes et générerSkipgrammes, qui sont toutes deux de complexité O(k * m^2) où k est le nombre de longueurs de N-Grammes/SkipGrammes et m est le nombre de lemmes.

fonction lireConllu(fichier) {
    liste phrases ← []
    liste phraseCourante ← []
    
    tant que non finDeFichier(fichier) {
        chaine ligne ← lireLigne(fichier)
        ligne ← ligne.trim()
        
        si ligne.commence_par("# text") {
            continuer
        }
        
        si ligne == "" {
            si phraseCourante != [] {
                phrases.ajouter(phraseCourante)
                phraseCourante ← []
            }
        } sinon {
            liste parties ← ligne.diviser("\t")
            si parties.taille >= 4 {
                dict token ← {
                    "id": parties[0],
                    "form": parties[1],
                    "lemma": parties[2],
                    "upos": parties[3],
                    "deprel": parties[6]
                }
                phraseCourante.ajouter(token)
            }
        }
    }
    
    si phraseCourante != [] {
        phrases.ajouter(phraseCourante)
    }
    retourner phrases
}

fonction validerConllu(phrases) {
    pour chaque phrase dans phrases {
        pour chaque token dans phrase {
            si non (token["id"] et token["form"] et token["lemma"] et token["upos"]) {
                lever erreur("Le fichier CONLLU est incorrect ou incomplet")
            }
        }
    }
}

fonction traiterPhrases(phrases) {
    entier nbTokens ← 0
    entier nbPhrases ← phrases.taille
    ensemble formes ← {}
    ensemble lemmes ← {}
    entier ponctuations ← 0
    dict comptePOS ← {}
    
    pour chaque phrase dans phrases {
        nbTokens ← nbTokens + phrase.taille
        pour chaque token dans phrase {
            formes.ajouter(token["form"])
            lemmes.ajouter(token["lemma"])
            
            si token["upos"] == "PUNCT" {
                ponctuations ← ponctuations + 1
            } sinon {
                comptePOS[token["upos"]][token["lemma"]] ← comptePOS[token["upos"]][token["lemma"]] + 1
            }
        }
    }
    
    dict stats ← {
        "nbTokens": nbTokens,
        "nbPhrases": nbPhrases,
        "nbFormes": formes.taille,
        "nbTypes": lemmes.taille,
        "ponctuations": ponctuations,
        "posDist": comptePOS
    }
    retourner stats
}

fonction générerNgrammes(lemmes, longueurMin, longueurMax) {
    dict ngrammes ← {}
    
    pour entier n de longueurMin à longueurMax {
        dict freqs ← {}
        entier i ← 0
        tant que i <= lemmes.taille - n {
            liste ngram ← lemmes[i:i+n]
            freqs[ngram] ← freqs[ngram] + 1
            i ← i + 1
        }
        ngrammes[n] ← freqs
    }
    retourner ngrammes.trierParFrequence()
}

fonction dédupliquerNgrammes(ngrams, seuil) {
    pour entier n de ngrams.taille à 2 par pas de -1 {
        si non ngrams[n-1] {
            continuer
        }
        pour chaque ngram dans ngrams[n] {
            liste candidats ← générerSousNgrammes(ngram)
            pour chaque candidat dans candidats {
                si ngrams[n-1][candidat] < seuil {
                    supprimer ngrams[n-1][candidat]
                }
            }
        }
    }
    retourner ngrams
}

fonction générerSkipgrammes(lemmes, intervalle, longueurMin, longueurMax) {
    dict skipgrammes ← {}
    
    pour entier n de longueurMin à longueurMax {
        dict freqs ← {}
        entier i ← 0
        tant que i <= lemmes.taille - n {
            entier gap ← 0
            tant que gap <= intervalle {
                liste skip ← extraireSkipgramme(lemmes, i, n, gap)
                freqs[skip] ← freqs[skip] + 1
                gap ← gap + 1
            }
            i ← i + 1
        }
        skipgrammes[n] ← freqs
    }
    retourner skipgrammes.trierParFrequence()
}

fonction principale(fichierConllu, fichierJson, fichierMotifs, seuil, ngramMin, ngramMax, intervalle) {
    liste phrases ← lireConllu(fichierConllu)
    validerConllu(phrases)
    
    dict statistiques ← traiterPhrases(phrases)
    liste lemmes ← extraireTousLemmes(phrases)
    
    dict ngrammes ← générerNgrammes(lemmes, ngramMin, ngramMax)
    statistiques["ngrams"] ← dédupliquerNgrammes(ngrammes, seuil)
    
    si fichierMotifs {
        liste motifs ← chargerMotifs(fichierMotifs)
        statistiques["patterns"] ← extraireMotifs(phrases, motifs)
    }
    
    statistiques["skipgrams"] ← générerSkipgrammes(lemmes, intervalle, ngramMin, ngramMax)
    écrireFichierJson(fichierJson, statistiques)
}